## 6. 작사가 인공지능 만들기

#### 목표 
- 텍스트 제너레이션 결과 그럴듯한 문장으로 생성하기
- 특수문자 제거, 토크나이저 생성, 패딩처리 등 전처리 과정
- 텍스트 생성모델의 validation loss 값 2.2 수준으로 달성

### (1) 라이브러리 import 

In [1]:
import glob
import os
import numpy as np
import pandas as pd
import tensorflow as tf

#효율적인 interator를 위한 툴 모듈
import itertools 
import re

from sklearn.model_selection import train_test_split
from tensorflow import keras

### (2) 데이터 읽어오기

In [2]:
# 여러개의 txt 파일을 모두 raw_corpus 에 담기 

txt_file_path = os.getenv('HOME')+'/aiffel//lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)
raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Busted flat in Baton Rouge, waitin' for a train", "And I's feelin' near as faded as my jeans", 'Bobby thumbed a diesel down, just before it rained']


In [3]:
# 중복된 문장들이 많은 파일들은 앞서 제거하는 함수

def Duplicate_f(file1, file2):
    txt_before = []
    txt_after = []
    with open(file1, "r", encoding="utf-8") as f:
        raw = f.read().splitlines()
        txt_before.extend(raw)
    with open(file2, "r", encoding="utf-8") as f:
        raw = f.read().splitlines()
        txt_after.extend(raw)
    txt_before = set(txt_before)
    txt_after = set(txt_after)
    diff = txt_before.difference(txt_after)
    return len(txt_before) * 0.05 > len(diff)

# 반복하면서 체크하고 해당되는 파일들을 리스트에서 제거
for a, b in itertools.combinations(txt_list, 2):
    if Duplicate_f(a, b):
        print(a, b)
        txt_list.remove(b)

/aiffel/aiffel//lyricist/data/lyrics/kanye-west.txt /aiffel/aiffel//lyricist/data/lyrics/Kanye_West.txt
/aiffel/aiffel//lyricist/data/lyrics/notorious-big.txt /aiffel/aiffel//lyricist/data/lyrics/notorious_big.txt


In [4]:
df = pd.DataFrame(raw_corpus)
df

,0
0,"Busted flat in Baton Rouge, waitin' for a train"
1,And I's feelin' near as faded as my jeans
2,"Bobby thumbed a diesel down, just before it ra..."
3,It rode us all the way to New Orleans I pulled...
4,"I was playin' soft while Bobby sang the blues,..."
...,...
187083,"1 2 3 4. Fiasco, fiasco [Chorus:]"
187084,1 2 3 4
187085,"I'm calling, I'm calling, I'm calling it fiasco,"
187086,"I'm calling, I'm calling, I'm calling it fiasco,"


In [5]:
# 제대로 추려졌는지 문장들을 출력해서 확인

for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue    # 길이가 0인 문장은 건너뜀
    if sentence[-1] == ":": continue   # 문장의 끝이 : 인 문장은 건너뜀

    if idx > 9: break   # 문장 10개 확인
        
    print(sentence)

Busted flat in Baton Rouge, waitin' for a train
And I's feelin' near as faded as my jeans
Bobby thumbed a diesel down, just before it rained
It rode us all the way to New Orleans I pulled my harpoon out of my dirty red bandanna
I was playin' soft while Bobby sang the blues, yeah
Windshield wipers slappin' time, I was holdin' Bobby's hand in mine
We sang every song that driver knew Freedom's just another word for nothin' left to lose
Nothin', don't mean nothin' hon' if it ain't free, no no
And, feelin' good was easy, Lord, when he sang the blues
You know, feelin' good was good enough for me


### (3) 데이터 정제

- Preprocess_sentence() 함수를 활용해 데이터를 정제
- 추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거
- 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가면 잘라내기 

In [6]:
# 정규표현식을 이용해서 문장들을 정리(특수문자 제거에 유의)
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1 소문자로 바꾸고, 양쪽 공백 삭제
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2 특수문자 양쪽에 공백 삽입
    sentence = re.sub(r'[" "]+', " ", sentence) # 3 여러개의 공백은 하나의 공백으로 교체
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4 a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 통일
    sentence = sentence.strip() # 5 다시 양쪽 공백 삭제
    sentence1 = sentence.split(' ')

    # 갯수가 14개 이하의 문장들만을 추려낸다
    if len(sentence1) > 13: 
        return 0
    else:
        sentence = '<start> ' + sentence + ' <end>' # 6 문장 시작에는 <start>, 끝에는 <end>를 추가
        return sentence
    
print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인

<start> this is sample sentence . <end>


In [7]:
# 앞서 정제된 문장들 중에서 필요한 문장들만 다시 리스트로 추려내기

corpus = []

for sentence in raw_corpus: # 반복하면서 원하지 않는 문장은 패스
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    if preprocessed_sentence ==0:
        pass
    else:
        corpus.append(preprocess_sentence(sentence))
                
# 졍제된 문장을 10개 정도만 확인
corpus[:10]

['<start> busted flat in baton rouge , waitin for a train <end>',
 '<start> and i s feelin near as faded as my jeans <end>',
 '<start> bobby thumbed a diesel down , just before it rained <end>',
 '<start> i was playin soft while bobby sang the blues , yeah <end>',
 '<start> windshield wipers slappin time , i was holdin bobby s hand in mine <end>',
 '<start> you know , feelin good was good enough for me <end>',
 '<start> there bobby shared the secrets of my soul <end>',
 '<start> through all kinds of weather , through everything we done <end>',
 '<start> he s lookin for that home , and i hope he finds it <end>',
 '<start> nothin , that s all that bobby left me , yeah <end>']

### (4) 평가 데이터셋 분리 

In [8]:
# 단어를 토큰화 시키기 

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=17000,  # 전체 단어의 개수 
        filters=' ',    
        oov_token="<unk>"  
    )
    
    
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됨

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공
    # maxlen의 디폴트값은 None - 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰짐
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
#     tensor = np.delete(tensor, slice(13, -1), axis=1)
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2 4009 1674 ...    0    0    0]
 [   2    8    4 ...    0    0    0]
 [   2  871 6826 ...    0    0    0]
 ...
 [   2   22   22 ...    3    0    0]
 [   2   22 4254 ...    0    0    0]
 [   2    3    0 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7feab21cc790>


In [9]:
# tensor 로 변환된 후의 크기를 확인

tensor.shape

(156013, 15)

In [10]:
# 제대로 바뀌었는지 확인을 위해 tensor 일부를 체크

print(tensor[:10, :10])

[[    2  4009  1674    14 12440  4486     5  1295    28     9]
 [    2     8     4    17   560   857    81  2711    81    13]
 [    2   871  6826     9  6263    58     5    32   174    11]
 [    2     4    53   998   924   230   871  2634     6  1119]
 [    2  5374  6827  4011    76     5     4    53  1141   871]
 [    2     7    34     5   560   115    53   115   255    28]
 [    2    62   871  3397     6  1730    20    13   300     3]
 [    2   128    25  3170    20  1675     5   128   177    21]
 [    2    54    17   457    28    15   159     5     8     4]
 [    2   497     5    15    17    25    15   871   243    12]]


In [11]:
# 확인 
len(tokenizer.index_word)

25663

In [12]:
'chorus' in tokenizer.index_word.values()

True

In [13]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 15: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to
11 : it
12 : me
13 : my
14 : in
15 : that


In [14]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성
# 여기서 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높음
src_input = tensor[:, :-1]

# tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input = tensor[:, 1:]
  
print(src_input[0])
print(tgt_input[0])

[    2  4009  1674    14 12440  4486     5  1295    28     9   636     3
     0     0]
[ 4009  1674    14 12440  4486     5  1295    28     9   636     3     0
     0     0]


### (5) 평가 데이터셋 분리

- testset 과 trainset 을 80% : 20% 으로 분리

In [15]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=42)

In [16]:
# 분리된 데이터 체크
print("- Train dataset:", enc_train.shape)
print("- Train label:", dec_train.shape)
print("- Validation dataset:", enc_val.shape)
print("- Validation label:", dec_val.shape)


- Train dataset: (124810, 14)
- Train label: (124810, 14)
- Validation dataset: (31203, 14)
- Validation label: (31203, 14)


- 분리가 된 것을 확인 

In [17]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124810, 14)
Target Train: (124810, 14)


In [18]:
print(enc_train[0])
print(dec_train[0])

[   2   14   15 1723    5   44 2044    5   43   57    5    7    1    3]
[  14   15 1723    5   44 2044    5   43   57    5    7    1    3    0]


In [19]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 128
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((128, 14), (128, 14)), types: (tf.int32, tf.int32)>

In [20]:
valid_ds = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
valid_ds = valid_ds.batch(BATCH_SIZE, drop_remainder=True)
valid_ds

<BatchDataset shapes: ((128, 14), (128, 14)), types: (tf.int32, tf.int32)>

### (6) 모델 만들기 

- embedding size, hidden size 를 조절
- Epoch 10 이하 
- validation loss 값을 2.2 이하로 달성 

In [21]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.batch_norm_1 = tf.keras.layers.BatchNormalization()
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.batch_norm_2 = tf.keras.layers.BatchNormalization()
        self.linear = tf.keras.layers.Dense(vocab_size)
        
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.batch_norm_1(out)
        out = self.rnn_2(out)
        out = self.batch_norm_2(out)
        out = self.linear(out)
        
        return out

# 조절 ----------------
embedding_size = 512
hidden_size = 1024
# ---------------------

model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [22]:
# 데이터셋에서 데이터 한 배치만 가져오기 

for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(128, 14, 17001), dtype=float32, numpy=
array([[[ 1.58492549e-04, -7.44797580e-05,  1.17044976e-04, ...,
          3.48531059e-04, -2.53725913e-04,  4.36148839e-05],
        [ 2.27430268e-04, -7.25518985e-05,  1.17454372e-04, ...,
          6.51283888e-04, -3.27402755e-04,  3.24467578e-06],
        [ 1.33342517e-04,  4.26046172e-05, -1.00617151e-04, ...,
          1.03721407e-03, -3.48332425e-04, -3.01056454e-04],
        ...,
        [-1.42928737e-03,  3.13553697e-04, -2.97523383e-03, ...,
          2.34041177e-03,  4.76527202e-04, -2.56053009e-03],
        [-1.53309223e-03,  2.86865281e-04, -3.14585562e-03, ...,
          2.32706964e-03,  5.96778176e-04, -2.65947590e-03],
        [-1.62325322e-03,  2.62316258e-04, -3.28620337e-03, ...,
          2.30668555e-03,  7.10810884e-04, -2.74861418e-03]],

       [[ 1.58492549e-04, -7.44797580e-05,  1.17044976e-04, ...,
          3.48531059e-04, -2.53725913e-04,  4.36148839e-05],
        [ 5.67907642e-04, -3.33910604e-04,  2

In [23]:
# 정보를 확인 
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  8704512   
_________________________________________________________________
lstm (LSTM)                  multiple                  6295552   
_________________________________________________________________
batch_normalization (BatchNo multiple                  4096      
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
batch_normalization_1 (Batch multiple                  4096      
_________________________________________________________________
dense (Dense)                multiple                  17426025  
Total params: 40,826,985
Trainable params: 40,822,889
Non-trainable params: 4,096
____________________________________

In [24]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, validation_data=valid_ds, epochs=10)

Epoch 1/10
975/975 [==============================] - 220s 223ms/step - loss: 3.2756 - val_loss: 2.8882
Epoch 2/10
975/975 [==============================] - 261s 267ms/step - loss: 2.7141 - val_loss: 2.6668
Epoch 3/10
975/975 [==============================] - 260s 267ms/step - loss: 2.4514 - val_loss: 2.5308
Epoch 4/10
975/975 [==============================] - 261s 268ms/step - loss: 2.2305 - val_loss: 2.4366
Epoch 5/10
975/975 [==============================] - 261s 267ms/step - loss: 2.0447 - val_loss: 2.3687
Epoch 6/10
975/975 [==============================] - 261s 267ms/step - loss: 1.8869 - val_loss: 2.3254
Epoch 7/10
975/975 [==============================] - 261s 267ms/step - loss: 1.7502 - val_loss: 2.2843
Epoch 8/10
975/975 [==============================] - 261s 268ms/step - loss: 1.6310 - val_loss: 2.2643
Epoch 9/10
975/975 [==============================] - 262s 268ms/step - loss: 1.5266 - val_loss: 2.2526
Epoch 10/10
975/975 [==============================] - 262s 268m

In [25]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
# 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    
# 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 

# 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줌
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

#모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

 
    generated = ""

# 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 최종적으로 모델이 생성한 자연어 문장

In [28]:
# text generation 결과 확인 

print(generate_text(model, tokenizer, init_sentence='<start> he', max_len=15))
print(generate_text(model, tokenizer, init_sentence='<start> she', max_len=15))
print(generate_text(model, tokenizer, init_sentence='<start> say', max_len=15))
print(generate_text(model, tokenizer, init_sentence='<start> everything', max_len=15))
print(generate_text(model, tokenizer, init_sentence='<start> anywhere', max_len=15))
print(generate_text(model, tokenizer, init_sentence='<start> somebody', max_len=15))
print(generate_text(model, tokenizer, init_sentence='<start> like', max_len=15))
print(generate_text(model, tokenizer, init_sentence='<start> why', max_len=15))
print(generate_text(model, tokenizer, init_sentence='<start> money', max_len=15))
print(generate_text(model, tokenizer, init_sentence='<start> love', max_len=15))

<start> he s a monster <end> 
<start> she s got me runnin round and round <end> 
<start> say it all <end> 
<start> everything i own i give you rocked my world <end> 
<start> anywhere you wanna <end> 
<start> somebody s out <end> 
<start> like a pony would like a pony would <end> 
<start> why do you love me <end> 
<start> money , money , money , money <end> 
<start> love is a losing game <end> 


## 회고

####  목표 : val_loss 2.2 수준으로 줄이기, epoch = 10 고정


[Epoch 10/10
975/975 [==============================] - 262s 268ms/step - loss: 1.4350 - val_loss: 2.2423]




- 이번 노드에서는 가사 데이터를 학습 : 특수문자 제거, 토크나이저 생성, 패딩 처리 등의 과정을 거침 
- Text Generator 결과는 그럴듯한 문장으로 생성 됨 예) 'why do you love me'
- 인공지능이 문장을 이해하는 방식과 작문을 가르치는 법을 배웠는데 Embedding size 와 hidden size 의 기본값만으로 성능은 잘 나와서 데이터의 질이 중요하다는 것을 다시한번 느낄수 있었음
 
